In [1]:
import numpy as np

data_path = './data/other/miscellaneous/piano/'

In [2]:
import os
import numpy
from keras.utils import np_utils
from keras import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
from keras.callbacks import ModelCheckpoint
import pretty_midi
from music21 import converter, instrument, note, stream, chord
from pathlib import Path

In [3]:
def get_files(path):
  fs = []
  files = Path(path).rglob('*.mid')
  for f in files:
    path = f.resolve()
    # print(path)
    fs.append(str(path))
  return fs

def get_notes(file):
  # print(file)
  notes = []
  try:
    mf = pretty_midi.PrettyMIDI(file)
    # print(mf)
    instrument = mf.instruments[0]
    # print(instrument.name)
    prev_note = instrument.notes[0]
    tmp = str(prev_note.pitch)
    for note in instrument.notes[1:]:
      if(prev_note.start == note.start):
        tmp += '.' + str(note.pitch)
      else:
        notes.append(tmp)
        tmp = str(note.pitch)
      prev_note = note
      notes.append(tmp)
  except Exception as error:
    print(error)
  return numpy.array(notes)

In [4]:
music_files = get_files(data_path)
print(len(music_files))

720


In [5]:
# notes_array = numpy.array([get_notes(i) for i in music_files[:300]], dtype=object)
notes_array = numpy.array([get_notes(i) for i in music_files], dtype=object)

/Users/bearsafari/Documents/ML/project/venv/lib/python3.9/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(
/Users/bearsafari/Documents/ML/project/venv/lib/python3.9/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


data byte must be in range 0..127


In [6]:
notes_ = [element for note_ in notes_array for element in note_]

In [7]:
n_vocab = len(set(notes_))
sequence_length = 50
# get all pitch names
pitchnames = sorted(set(item for item in notes_))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes_) - sequence_length, 1):
    sequence_in = notes_[i:i + sequence_length]
    sequence_out = notes_[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input_norm = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

In [8]:
from keras.utils import Sequence

class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x = x_set
        self.y = y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(numpy.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        x_batch = self.x[idx * self.batch_size:(idx+1) * self.batch_size]
        y_batch = self.y[idx * self.batch_size:(idx+1) * self.batch_size]
        return x_batch, y_batch

batch_size = 512

network_input_gen = DataGenerator(network_input_norm, network_output, batch_size)

In [9]:
model = Sequential()
model.add(LSTM(
    512,
    input_shape=(network_input_norm.shape[1], network_input_norm.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

Metal device set to: Apple M1


In [ ]:
filepath = "./weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,
    mode='min'
)    
callbacks_list = [checkpoint]     
model.load_weights("./weights-improvement-13-3.3430-bigger.hdf5")
model.fit(network_input_gen, epochs=200, batch_size=batch_size, callbacks=callbacks_list)

Epoch 21/200


2023-05-24 00:47:41.330166: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1072/1114 [===========================>..] - ETA: 29s - loss: 3.5085

In [14]:
model = Sequential()
model.add(LSTM(
    512,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
# Load the weights to each node
f = "./weights/weights-improvement-126-2.2941-bigger.hdf5"
model.load_weights(f)

In [44]:
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
print(start)
pattern = network_input[start]
# print(pattern)
prediction_output = []
# generate 500 notes
last_index = 0
for note_index in range(200):
    # print(pattern)
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    # print("pred input", prediction_input)
    prediction = model.predict(prediction_input, verbose=0)
    # print(numpy.sum(prediction))

    # index = numpy.argmax(prediction)
    # if index == last_index:
    index = numpy.argsort(numpy.max(prediction, axis=0))[-numpy.random.randint(1, 4)]
    last_index = index
    result = int_to_note[index]
    # print(index, result)
    prediction_output.append(result)
    pattern = numpy.append(pattern, index)
    pattern = pattern[1:len(pattern)+1]

68481


In [45]:
from music21 import converter, instrument, note, stream, chord

offset = 0
output_notes = []
print(prediction_output)
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

['50', '45', '50', '54', '57.62', '57', '62', '62', '69', '69.74', '69.81', '66', '66', '62', '69', '68', '65', '65.77', '43', '43', '72', '81', '78', '57', '57', '62', '60', '62', '60', '55', '56', '79', '79', '67', '68', '88', '88', '91', '95', '93', '95', '88', '88', '86', '85', '88', '85', '86', '88', '85', '86', '86', '86', '85', '85', '82', '82', '82', '83', '83', '84', '82', '55', '53', '50', '50', '51', '50', '79', '48', '48', '55', '55', '55', '50', '45', '48', '41', '45', '53', '54', '53', '46', '49', '47', '41', '47', '51', '51', '47', '47', '51', '50', '54', '54', '57', '57', '54', '55', '55', '52', '51', '52', '52', '48', '48', '51', '51', '54', '51', '54', '51', '51', '52', '52', '50', '48', '51', '50', '50', '50', '51', '51', '50', '50', '51', '53', '51', '50', '50', '51', '51', '48', '48', '48', '50', '48', '47', '50', '51', '48', '45', '45', '45', '47', '50', '50', '48', '47', '48', '51', '52', '52', '50', '47', '49', '51', '52.55', '50', '48', '47', '52', '52', '47', 

In [46]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output12.mid')

'test_output12.mid'